In [1]:
import pandas as pd
import csv
import os
import time
from collections import defaultdict
import numpy as np
import itertools
import db_structure
import networkx as nx
from imp import reload
import matplotlib.pyplot as plt
import utilities as u
from flask import jsonify
import logging
# import exceptions
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype

logging.basicConfig(format=' %(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
%load_ext autoreload
%aimport -web
%autoreload 2

 2019-10-11 14:08:00,457 - DEBUG - backend module://ipykernel.pylab.backend_inline version unknown


In [31]:
db = db_structure.DB(os.path.join('datasets', 'TOPICC'))
dm = db_structure.DataManager(db)

 2019-10-11 14:16:19,468 - INFO - Loading datasets\TOPICC
 2019-10-11 14:16:19,479 - INFO - Loading arch file: datasets\TOPICC\TOPICC.arch


In [39]:
all_chosen_idxs = ['260', '294']
paths = db.find_paths_multi_columns(all_chosen_idxs)
df = dm.get_biggest_joined_df_option_from_paths(paths, filter_col_idxs=all_chosen_idxs)

 2019-10-11 14:21:46,504 - DEBUG - Loading file datasets\TOPICC\DEATH.csv


In [40]:
print(len(df))
df.head()

58


,CardiacSurgTime_[CARDIACSURGERY],DeathMode_[DEATH]
0,1413,Withdrawal of care
1,758,Failed resuscitation
2,732,Withdrawal of care
3,715,Failed resuscitation
4,919,Withdrawal of care


In [41]:
filters =  {'260': {'type': 'range', 'filter': {'min': 27, 'max': 2306, 'bins': '10'}}}

In [42]:
filters = dm.rewrite_filters(filters)

In [43]:
filters

{'260': {'type': 'bins',
  'filter': [27.0,
   254.9,
   482.8,
   710.7,
   938.6,
   1166.5,
   1394.4,
   1622.3,
   1850.2,
   2078.1,
   2306.0]}}

In [44]:
new_df = dm.filter_df(df, filters)

In [56]:
new_df.groupby('CardiacSurgTime_[CARDIACSURGERY]', observed=True)['DeathMode_[DEATH]'].value_counts()

CardiacSurgTime_[CARDIACSURGERY]  DeathMode_[DEATH]   
(710.7, 938.6)                    Withdrawal of care      19
                                  Failed resuscitation    14
                                  Limitation of care       1
(938.6, 1166.5)                   Failed resuscitation     3
                                  Withdrawal of care       3
                                  Limitation of care       1
(1166.5, 1394.4)                  Withdrawal of care       6
                                  Failed resuscitation     5
                                  Brain death              1
(1394.4, 1622.3)                  Withdrawal of care       4
(1622.3, 1850.2)                  Failed resuscitation     1
Name: DeathMode_[DEATH], dtype: int64

In [ ]:
newer_df = dm.aggregate_df(df=new_df, groupby_col_idxs=groupby_col_idxs, filters=filters)

In [ ]:
from decimal import Decimal as D
x = D(123.55235235423)
pre_dec, post_dec = str(x).split('.')
if len(post_dec) > 2:
    x = D(pre_dec + '.' + post_dec[:2])
x

In [ ]:
x = pd.DataFrame(columns=['a'])
x['b'] = None
x

In [ ]:
draw_db = db

G = nx.DiGraph()
G.add_nodes_from(draw_db.table_names)
for table_name in draw_db.table_names:
    for sibling in draw_db.find_table_siblings(table_name):
        G.add_edge(table_name, sibling)
        G.add_edge(sibling, table_name)
    for child in draw_db.find_table_children(table_name):
        G.add_edge(table_name, child)
plt.figure(figsize=(10,10))

#draw with networkx built-ins
nx.draw_networkx(G, node_shape="None", width=0.2)

#OR

#draw using pygraphviz
#A = nx.nx_agraph.to_agraph(G)
#H = nx.nx_agraph.from_agraph(A)
#nx.draw_spring(H, node_shape="None", with_labels=True, width=0.2)